In [ ]:
import pandas as pd
import numpy as np
import os
import requests
import datetime
from datetime import date, datetime, timedelta
from PyPDF2 import PdfReader, PdfWriter
import holidays
import calendar
from collections import Counter
import tkinter as tk
from tkinter import messagebox

In [ ]:
# Extracting Clockify workspace ID (for the summary report)
url_base = 'https://api.clockify.me/api/v1/user'
API_KEY = os.environ['Clockify_API_Key']

headers = {'content-type': 'application/json', 'X-Api-Key': API_KEY}
response = requests.get(url_base, headers=headers)
json_response_base = response.json()
json_response_base['id']

workspace_id = json_response_base['activeWorkspace']

In [ ]:
# Getting current day, month, and year
today = date.today()
current_day = today.day
current_month = today.month
current_year = today.year
current_year_short = str(current_year)[-2:] # Last two digits of the year

# Getting first and last day of the previous month (date)
current_first = today.replace(day=1)
prev_last = current_first - timedelta(days=1)
prev_first = prev_last.replace(day=1)
prev_first_str = str(prev_first) # First day of the previous month (date)
prev_last_str = str(prev_last) # Last day of the previous month (date)

# Getting previous month and year
last_date = prev_first.strftime("%Y-%m")
split_date = last_date.split("-")
month_prev = split_date[1]
month_prev_int = int(month_prev) # Previous month
year_prev = split_date[0]
year_prev_int = int(year_prev) # Year of the previous month
year_prev_short = year_prev[-2:] # Last two digits of the year

In [ ]:
# Function to get the Clockify summary report for the previous month
def get_summary_report():
    url = f"https://reports.api.clockify.me/v1/workspaces/"+workspace_id+"/reports/summary"
    headers = {'X-Api-Key': API_KEY}
    data = {
        "dateRangeStart": prev_first_str+"T00:00:00",
        "dateRangeEnd": prev_last_str+"T23:59:59",
        "summaryFilter": {
            "groups": [
             "USER",
             "DATE"
            ]
        }
        }

    return requests.post(url, headers=headers, json=data).json()

In [ ]:
# Getting the summary report
summary_report = get_summary_report()

In [ ]:
# JSON to dataframe
dataframe = pd.json_normalize(summary_report, record_path=['groupOne', 'children'], meta=[['children', 'name']])

# Rename columns
dataframe.rename(columns={'name':'date', 'children.name':'name'}, inplace=True)

In [ ]:
# Create a list of all dates in the last month
date_list = [prev_last - timedelta(days=x) for x in range(prev_last.day)]

# Create a dataframe with all the dates in the last month
names = dataframe['name'].unique().tolist()
df_dates = pd.DataFrame({'name':[], 'date':[]})

# Create a dataframe with all the dates and names
for name in names:
    df_name = pd.DataFrame({'name':[name]*len(date_list), 'date':date_list})
    df_dates = pd.concat([df_dates, df_name])

# Convert the date column to datetime
dataframe['date'] = pd.to_datetime(dataframe['date'])
df_dates['date'] = pd.to_datetime(df_dates['date'])

# Merge the two dataframes
dataframe = pd.merge(df_dates, dataframe, on=['name', 'date'], how='left')

# Fill in the missing values with NaN
dataframe['duration'] = dataframe['duration'].replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# Calculate duration in hours (decimal form)
dataframe['hours'] = dataframe['duration']/3600
dataframe['hours'] = dataframe['hours'].apply(lambda x:round(x,2)) # Round to 2 decimal places

# Calculate total duration per person
total_hours = dataframe.groupby(['name'])['hours'].transform('sum')
dataframe['total_hours'] = total_hours
dataframe['total_hours'] = dataframe['total_hours'].round(2) # Round to 2 decimal places

# Drop unnecessary columns
dataframe = dataframe.drop(columns=['amounts', 'amount', '_id', 'duration'])

# Create a dictionary with the total hours per person (before the changes are applied)
total_hours_dict_before = dict(zip(dataframe['name'], dataframe['total_hours']))

In [ ]:
# Get holidays in the Netherlands
holidaysNL = holidays.NL(years=year_prev_int)

# Get the last day of the previous month
last_day = calendar.monthrange(year_prev_int, month_prev_int)[1]

In [ ]:
# Function to adjust the hours worked on holidays and weekends
def adjust_hours(df):
    # Sort the dataframe by name and date
    df = df.sort_values(by=['name', 'date']).reset_index(drop=True)

    # Create a dictionary to store the hours worked on each day
    hours_per_day = {}

    # Loop through each row in the dataframe
    for i, row in df.iterrows():
        name = row['name']
        date = row['date']
        hours = row['hours']
        date_dt = pd.to_datetime(date).date()
        day = date_dt.day
        
        if (pd.isnull(df.loc[(df['name'] == name) & (df['date'] == date), 'hours']).all()):
            # If the person didn't work on that day, skip to the next row
            continue

        elif ((date.weekday() < 5) and (date not in holidaysNL)):
            # If it's not a weekday or holiday, add the working hours to the dictionary
            if (date, name) in hours_per_day:
                hours_per_day[(date, name)] += hours
            else:    
                hours_per_day[(date, name)] = hours

        else:
            # Calculate the next available business day
            next_weekday = date
            while ((next_weekday, name) in hours_per_day # If the date is already in the dictionary
                or (next_weekday.weekday() >= 5) # If it's a weekend
                or (next_weekday in holidaysNL) # If it's a holiday
                or (pd.notnull(df.loc[(df['name'] == name) & (df['date'] == next_weekday), 'hours']).all())): # If the person already worked on that day
                next_weekday += timedelta(days=1) # Add one day
                
                # If we've reached the end of the month, check backwards
                if next_weekday.month != date.month:
                    # Calculate the previous available business day
                    previous_weekday = date
                    while ((previous_weekday, name) in hours_per_day # If the date is already in the dictionary
                        or (previous_weekday.weekday() >= 5) # If it's a weekend
                        or (previous_weekday in holidaysNL) # If it's a holiday
                        or (pd.notnull(df.loc[(df['name'] == name) & (df['date'] == previous_weekday), 'hours']).all())): # If the person already worked on that day
                        previous_weekday -= timedelta(days=1) # Subtract one day

                        # If we've reached the beginning of the month, get the next business day
                        if previous_weekday.month != date.month:
                            print(f"No available weekday found for {name} on {date}...")
                            # Calculate the next business day
                            next_day = date
                            while (next_day.weekday() >= 5 or next_day in holidaysNL):
                                next_day += timedelta(days=1)
                            # If we've reached the end of the month, check backwards    
                            if next_day.month != date.month:
                                # Calculate the previous business day
                                next_day = date
                                while (next_day.weekday() >= 5 or next_day in holidaysNL):
                                    next_day -= timedelta(days=1)
                                # If we've reached the beginning of the month, keep the hours on the original date
                                if next_day.month != date.month:
                                    next_day = date
                                    # Add the hours to the dictionary
                                    print(f"...keeping {hours} hours on {date} for {name}")
                                    if (next_day, name) in hours_per_day:
                                        hours_per_day[(next_day, name)] += hours
                                    else:
                                        hours_per_day[(next_day, name)] = hours
                                    break

                                else:
                                    # Add the hours to the dictionary
                                    print(f"...moving {hours} hours backward, from {date} to {next_day}, for {name}")
                                    if (next_day, name) in hours_per_day:
                                        hours_per_day[(next_day, name)] += hours
                                    else:
                                        hours_per_day[(next_day, name)] = hours
                                    break    
                            else:
                                # Add the hours to the dictionary
                                print(f"...moving {hours} hours forward, from {date} to {next_day}, for {name}")
                                if (next_day, name) in hours_per_day:
                                    hours_per_day[(next_day, name)] += hours
                                else:
                                    hours_per_day[(next_day, name)] = hours
                                break

                    # If we've reached the beginning of the month, break out of the loop    
                    if previous_weekday.month != date.month:
                        break

                    print(f"Moving {hours} hours backward, from {date} to {previous_weekday}, for {name}")
                    
                    # Add the hours to the dictionary
                    if (previous_weekday, name) in hours_per_day:
                        hours_per_day[(previous_weekday, name)] += hours
                    else:    
                        hours_per_day[(previous_weekday, name)] = hours
                    break

            # If we've reached the end of the month, break out of the loop
            if next_weekday.month != date.month:
                continue

            else:
                print(f"Moving {hours} hours forward, from {date} to {next_weekday}, for {name}")

                # Add the hours to the dictionary
                if (next_weekday, name) in hours_per_day:
                    hours_per_day[(next_weekday, name)] += hours
                else:    
                    hours_per_day[(next_weekday, name)] = hours
                
    return hours_per_day

In [ ]:
# Apply the function to the dataframem to get the dictionary with the adjusted hours
dictionary = adjust_hours(dataframe)

# Convert the dictionary to a dataframe
adjusted_df = pd.DataFrame([(k[0].date(), k[1], v) for k, v in dictionary.items()], columns=['date', 'name', 'hours'])

In [ ]:
# Create a dictionary with the total hours per person after changes
adjusted_df['total_hours'] = adjusted_df.groupby(['name'])['hours'].transform('sum')
adjusted_df['total_hours'] = adjusted_df['total_hours'].round(2) # Round to 2 decimal places
total_hours_dict_after = dict(zip(adjusted_df['name'], adjusted_df['total_hours']))

# Check if total hours before and after changes are equal
if Counter(total_hours_dict_before) == Counter(total_hours_dict_after):
    print("Total hours before and after changes are equal. Continuing with the code...")
else:
    root = tk.Tk()
    root.wm_attributes("-topmost", 1)
    root.withdraw()
    tk.messagebox.showerror("Error", "Something went wrong. The total number of hours per person is incorrect.", parent=root)
    root.destroy()

In [ ]:
# Sort the dataframe by name and date
adjusted_df = adjusted_df.sort_values(['name', 'date'], ascending=[True, True])

# Add the total hours to the dataframe
adjusted_df['total_hours'] = adjusted_df['name'].map(total_hours_dict_before)

# Get first name and last name, and initial
sep_names = adjusted_df['name'].str.split(' ', expand=True)
adjusted_df['first_name'] = sep_names[0]
adjusted_df['last_name'] = sep_names[1]
adjusted_df['last_name2'] = sep_names[2] # For people with two last names
adjusted_df['initial'] = adjusted_df['first_name'].astype(str).str[0]

# Extract the day from the date
date_split = adjusted_df['date'].astype('str').str.split("-", expand=True)
adjusted_df['day'] = date_split[2]

# Fill the empty cells with empty strings
adjusted_df.fillna("", inplace=True)

In [ ]:
# Group dataframe by name
dfs = [y for x, y in adjusted_df.groupby('name')]

# Get the number of people
people = len(dfs)

# Loop through the number of people
for x in range(people):
    # Get the dataframe for each person
    df = pd.DataFrame(dfs[x])
    
    # Create a dictionary with the day as key and the duration as value
    dict_of_hours = dict(zip(df.day, df.hours))
    
    # Get variables needed to fill in the forms and remove brackets and apostrophes
    first_name = df['first_name'].unique()
    first_name = str(first_name).replace("'", "").replace("[", "").replace("]", "")
    last_name = df['last_name'].unique()
    last_name = str(last_name).replace("'", "").replace("[", "").replace("]", "")
    last_name2 = df['last_name2'].unique()
    last_name2 = str(last_name2).replace("'", "").replace("[", "").replace("]", "")
    
    # Function to add a space before the last name if the person has a second last name
    def last_name2_field(last_name2):
        if last_name2 == "":
            return last_name2
        else:
            return ' '+last_name2
    
    # Function to add an underscore before the last name if the person has a second last name
    def last_name2_title(last_name2):
        if last_name2 == "":
            return last_name2
        else:
            return '_'+last_name2        
     
    last_name2_field = last_name2_field(last_name2) # For the field in the PDF file
    last_name2_title = last_name2_title(last_name2) # For the title of the PDF file
    
    initial = df['initial'].unique()
    initial = str(initial).replace("'", "").replace("[", "").replace("]", "")
    
    total_hours = df['total_hours'].unique()
    total_hours = str(total_hours).replace("[", "").replace("]", "")
    
    # Define reader and writer
    reader = PdfReader("tsh_form.pdf")
    writer = PdfWriter()
    
    # Read pages from the PDF file
    page1 = reader.pages[0]
    page2 = reader.pages[1]

    # Define the supervisor signing the form
    supervisor = 'Hannes Datta'

    # Define the Faculty / Office
    faculty = 'Tilburg School of Economics and Management'

    # Define the Department
    department = 'Marketing (Tilburg Science Hub)'
    
    # Fill in the fields in the PDF file
    writer.update_page_form_field_values(page1, fields=dict_of_hours)
    writer.update_page_form_field_values(page1, {'Naam en voorletters': last_name+last_name2_field+' '+initial, 
                                             'Faculteit  Dienst': faculty,
                                             'Departement  Afdeling': department,
                                             'maand1': month_prev, 
                                             'jaar1': year_prev_short,
                                             'tot': total_hours})
    writer.update_page_form_field_values(page2, {'dag1': current_day,
                                             'maand2': current_month,
                                             'jaar2': current_year_short,
                                             'Naam gemandateerde leidinggevende': supervisor,
                                             'dag3': current_day,
                                             'maand3': current_month,
                                             'jaar3': current_year_short})
    
    # Add pages
    writer.add_page(page1)
    writer.add_page(page2)
    
    # Save the PDF file
    save_path = './generated_forms/'
    filename = initial+'_'+last_name+last_name2_title+'_'+last_date
    completeName = os.path.join(save_path, filename+".pdf")
    with open(completeName,"wb") as new:
        writer.write(new)